<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/tech16_hw5_agents_all_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install "crewai[tools]"

In [ ]:
!pip install llama-index --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
!wget "https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf" -O GS_SnP500.pdf

--2025-03-04 04:47:05--  https://privatewealth.goldmansachs.com/outlook/2025-isg-outlook.pdf
Resolving privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)... 23.223.211.8, 23.223.211.33
Connecting to privatewealth.goldmansachs.com (privatewealth.goldmansachs.com)|23.223.211.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘GS_SnP500.pdf’

GS_SnP500.pdf           [  <=>               ]   6.88M  24.4MB/s    in 0.3s    

2025-03-04 04:47:05 (24.4 MB/s) - ‘GS_SnP500.pdf’ saved [7209986]



In [ ]:
!wget "https://am.jpmorgan.com/content/dam/jpm-am-aem/global/en/2025%20Year-Ahead%20Investment%20Outlook.pdf" -O JPM_Outlook.pdf

--2025-03-04 04:47:20--  https://am.jpmorgan.com/content/dam/jpm-am-aem/global/en/2025%20Year-Ahead%20Investment%20Outlook.pdf
Resolving am.jpmorgan.com (am.jpmorgan.com)... 146.143.82.3
Connecting to am.jpmorgan.com (am.jpmorgan.com)|146.143.82.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘JPM_Outlook.pdf’

JPM_Outlook.pdf         [      <=>           ]   1.44M  1.23MB/s    in 1.2s    

2025-03-04 04:47:22 (1.23 MB/s) - ‘JPM_Outlook.pdf’ saved [1512586]



In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI


reader = SimpleDirectoryReader(input_files=["GS_SnP500.pdf"])
docs = reader.load_data()
reader = SimpleDirectoryReader(input_files=["JPM_Outlook.pdf"])
docs2 = reader.load_data()

In [ ]:
docs[117].get_content()

'Goldman Sachs\nAmsterdam\nAtlanta\nAustin\nBeijing\nBoston\nChicago\nDallas\nDenver\nDetroit\nDubai\nDublin\nFrankfurt\nGeneva\nHong Kong\nHouston\nLondon\nLos Angeles\nLuxembourg\nMadrid\nMelbourne\nMiami\nMilan\nMonaco\nNashville / Brentwood\nNew York\nParis\nPhiladelphia\nSan Francisco\nSeattle\nShanghai\nShenzhen\nSingapore\nStockholm\nSydney\nTel Aviv\nWashington, D.C.\nWest Palm Beach\nZurich\nwww.gs.com'

In [ ]:
print(type(docs))
print(len(docs))

<class 'list'>
118


In [ ]:
print(type(docs2))
print(len(docs2))

<class 'list'>
16


In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")
index = VectorStoreIndex.from_documents(docs)
index = VectorStoreIndex.from_documents(docs2)
query_engine = index.as_query_engine(similarity_top_k=5,llm=llm)

/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)
/usr/local/lib/python3.11/dist-packages/llama_index/core/indices/base.py:110: DeprecationWarning: Call to deprecated method get_doc_id. ('get_doc_id' is deprecated, access the 'id_' property instead.) -- Deprecated since version 0.12.2.
  docstore.set_document_hash(doc.get_doc_id(), doc.hash)


In [ ]:
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="Global Outlook Query Tool",
    description="Use this tool to lookup the 2025 global finance outlook of GS and JPM"
)

In [ ]:
query_tool.args_schema.schema()

<ipython-input-31-fe53991c08af>:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  query_tool.args_schema.schema()


{'description': 'Schema for query tool.',
 'properties': {'query': {'description': 'Search query for the query tool.',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'QueryToolSchema',
 'type': 'object'}

# Simple example

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, LlamaIndexTool
from IPython.display import Markdown, display

# Define the research topic
topic = "What are JPM and GS 2025 outlook for the S&P 500 index?"

# Create the Researcher agent
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), query_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Assemble the crew and run tasks
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential
)

result = crew.kickoff()

# Display the final report
display(Markdown(result.raw))


# Agent: Financial Research Analyst
## Task: Conduct a comprehensive analysist of 'What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?' and summarize key findings and risk factors in 2025.


# Agent: Financial Research Analyst
## Thought: I need to gather insights from Goldman Sachs (GS) and JPMorgan (JPM) regarding their views on the 2025 outlook for the S&P 500 index. To do this, I will utilize the Global Outlook Query Tool to get the necessary information from both institutions.
## Using tool: Global Outlook Query Tool
## Tool Input: 
"{\"query\": \"2025 outlook S&P 500 site:gs.com\"}"
## Tool Output: 
The S&P 500 outlook for 2025 suggests that earnings growth for the Magnificent 7 companies is expected to decelerate, while the rest of the market is anticipated to catch up. This shift in earnings growth is projected to lead to a more inclusive rally, with a focus on factors such as economic resiliency, revenue gr

# Outlook for the S&P 500 in 2025: A Comparison of Goldman Sachs and JPMorgan

## Introduction
As we look ahead to 2025, the S&P 500 Index serves as a crucial barometer for the broader U.S. equity market. While both Goldman Sachs (GS) and JPMorgan (JPM) present optimistic projections for the index, their analyses differ significantly, particularly in terms of potential catalysts for growth and the associated risks. This report aims to compare and contrast their viewpoints, emphasizing the headwinds the index may encounter in the coming years.

## Goldman Sachs (GS) Outlook for 2025
Goldman Sachs offers a nuanced perspective on the future of the S&P 500, emphasizing the following key points:

- **Deceleration in Earnings Growth**: The anticipated slowdown in earnings growth for the prominent "Magnificent 7" companies (which includes tech giants like Apple and Microsoft) is a critical consideration. Advisors predict that the vigorous growth seen in previous years may not sustain its pace.

- **Catch-Up Effect**: GS highlights a potential "catch-up effect" from smaller companies in the index. As larger firms show signs of slowing growth, the broader market may witness an inclusive rally fueled by the performance of mid- and small-cap stocks. This shift could lead to a more balanced market performance.

- **Secular Trends**: Key macroeconomic and secular trends are expected to provide support to various sectors. These include increased spending on artificial intelligence, shifts in energy policies towards sustainable sources, and diversification in global supply chains. Such factors may invigorate sectors that lag in performance as technology adoption moves across industries.

## JPMorgan (JPM) Outlook for 2025
On the other hand, JPMorgan’s analysis underscores the prevailing influence of large-cap companies:

- **S&P 500 as a Market Gauge**: JPM asserts that the S&P 500 Index is the best indicator of the U.S. equities market. The index's composition reflects a substantial share of total market value, highlighting the importance of its largest constituents in shaping overall market performance.

- **Robustness of Large-Cap Stocks**: The focus remains squarely on the strength of large-cap companies within the index. JPM suggests that these firms will serve as pivotal players in navigating the economic landscape, potentially driving gains and providing stability.

## Key Comparisons
Upon examining both outlooks, several comparisons can be drawn:

- **Emphasis on Growth Catalysts**: Both firms acknowledge the importance of large-cap companies, but GS leans towards the positive dynamics seen in smaller firms and sector-specific growth drivers (like AI), whereas JPM focuses on large-cap robustness as the foundation of market strength.

- **Earnings Concerns**: Both analysts recognize the risks associated with decelerating earnings, particularly within leading technology firms. However, GS accentuates this concern more prominently, underscoring the significance of smaller companies in offsetting larger firms' performance.

## Risk Factors to Consider
As with any market forecast, both outlooks introduce particular risks that could impact the S&P 500:

1. **Earnings Deceleration**: Continued slowing growth in earnings, particularly among influential tech stocks, could dampen market performance.

2. **Economic Policy Changes**: Shifts in economic policy or regulatory environments could have far-reaching effects on large-cap firms, potentially leading to volatility.

3. **Global Economic Influences**: External global economic conditions may impinge upon U.S. market stability, affecting investor sentiment and market dynamics.

4. **Geopolitical Risks**: Heightened volatility stemming from geopolitical events may create uncertainty, adversely impacting investor confidence and market conditions.

## Conclusion
In summation, both Goldman Sachs and JPMorgan maintain an optimistic outlook for the S&P 500 in 2025 while presenting differing perspectives on the potential catalysts behind this growth. Goldman Sachs emphasizes the essential role of smaller companies and key secular trends, whereas JPMorgan holds large-cap firms as the pillars of market stability. Investors should remain cognizant of the associated risks, as external factors and internal dynamics could significantly influence market performance over the next few years. The interplay between these elements will define the trajectory of the S&P 500 as we approach 2025.

# More detailed

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
# Set your research topic and task-specific descriptions.
topic = "What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?"
# research_description = (
#     f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
# )
# research_expected_output = (
#     f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
# )

# writing_description = (
#     f"Compose an analysis report summarizing the findings on '{research_topic}'. "
#     "The report should be structured, easy to read, and include clear headings and bullet points."
# )
# writing_expected_output = (
#     f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
# )

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
# search_tool = SerperDevTool()        # Enables web searching via the Serper API
# web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages


# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), llama_chroma_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]




# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
# research_task = Task(
#     description=research_description,
#     expected_output=research_expected_output,
#     agent=researcher_agent
# )

# --------------------------
# Define the writing/summarization task for the Writer agent.
# --------------------------
# write_task = Task(
#     description=writing_description,
#     expected_output=writing_expected_output,
#     agent=writer_agent,
#     context=[research_task]  # This task uses the output of the research_task as context
# )

# --------------------------
# Assemble the crew with both agents and tasks.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent],
    tasks=[research_task, write_task],
    process=Process.sequential,  # Runs the research task before the writing task
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final analysis report as markdown
display(Markdown(result.raw))

SyntaxError: '(' was never closed (<ipython-input-55-7a8945299df5>, line 62)

### Include critic - Sequential

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
topic = "What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?"

critic_description = (
    f"Review the financial analysis report on the outlook of the S&P 500 and summarizing the findings on '{topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# # --------------------------
# # Set up the tools for the Researcher agent: a web search tool and a web content reader.
# # --------------------------
# search_tool = SerperDevTool()         # Enables web searching via the Serper API
# web_reader_tool = WebsiteSearchTool()   # Retrieves and reads content from web pages

# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), llama_chroma_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# Create the Critic agent
critic_agent = Agent(
    name="Financial Analysis Critic",
    role="Financial Content Critic",
    goal="Review and provide constructive feedback on the analysis report",
    backstory="An expert in critically evaluating financial content to ensure quality, accuracy, and clarity.",
    tools=[],  # No special tools needed for this critique task
    verbose=True
)


# Create the Rewriter agent
rewriter_agent = Agent(
    name="Rewriter",
    role="Content Editor",
    goal="Rewrite the analysis report by incorporating the critic's feedback",
    backstory="A skilled editor who refines content to enhance clarity, coherence, and overall quality.",
    tools=[],  # No special tools needed for this rewriting task
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Define the critic task for the Critic agent.
critic_task = Task(
    description=critic_description,
    expected_output=critic_expected_output,
    agent=critic_agent,
    context=[write_task]  # This task uses the output of the write_task as context
)

# Define the rewriting task for the Rewriter agent.
rewriter_task = Task(
    description=rewriter_description,
    expected_output=rewriter_expected_output,
    agent=rewriter_agent,
    context=[research_task, write_task, critic_task]  # This task uses both the original report and the critique as context
)

# --------------------------
# Assemble the crew with all four agents and tasks.
# --------------------------
research_crew = Crew(
    agents=[researcher, writer, critic_agent, rewriter_agent],
    tasks=[research_task, write_task, critic_task, rewriter_task],
    process=Process.sequential,  # Executes tasks in sequence: research -> writing -> critique -> rewrite
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))


# Agent: Financial Research Analyst
## Task: Conduct a comprehensive analysist of 'What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?' and summarize key findings and risk factors in 2025.


# Agent: Financial Research Analyst
## Thought: I need to gather information on the 2025 outlook for the S&P 500 index from both JPMorgan (JPM) and Goldman Sachs (GS) using the available tools. I will use the Global Outlook Query Tool to find insights related to the global finance outlook.
## Using tool: Global Outlook Query Tool
## Tool Input: 
"{\"query\": \"2025 outlook S&P 500 index site:Goldman Sachs\"}"
## Tool Output: 
The S&P 500 Index is widely regarded as the best single gauge of the U.S. equities market. The index includes a representative sample of 500 leading companies in leading industries of the U.S. economy. The S&P 500 Index focuses on the large-cap segment of the market; however, since it includes a significant

###Include Critic - Hierarchical

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# --------------------------
# Configuration for a generic research task
# --------------------------
topic = "What are the view of the overal market compared and contrasted to JPM and GS about the 2025 outlook for the S&P 500 index?"

critic_description = (
    f"Review the financial analysis report on the outlook of the S&P 500 and summarizing the findings on '{topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# # --------------------------
# # Set up the tools for the Researcher agent: a web search tool and a web content reader.
# # --------------------------
# search_tool = SerperDevTool()         # Enables web searching via the Serper API
# web_reader_tool = WebsiteSearchTool()   # Retrieves and reads content from web pages

# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), llama_chroma_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# Create the Critic agent
critic_agent = Agent(
    name="Financial Analysis Critic",
    role="Financial Content Critic",
    goal="Review and provide constructive feedback on the analysis report",
    backstory="An expert in critically evaluating financial content to ensure quality, accuracy, and clarity.",
    tools=[],  # No special tools needed for this critique task
    verbose=True
)


# Create the Rewriter agent
rewriter_agent = Agent(
    name="Rewriter",
    role="Content Editor",
    goal="Rewrite the analysis report by incorporating the critic's feedback",
    backstory="A skilled editor who refines content to enhance clarity, coherence, and overall quality.",
    tools=[],  # No special tools needed for this rewriting task
    verbose=True
)

# Define tasks
research_task = Task(
    description=f"Conduct a comprehensive analysist of '{topic}' and summarize key findings and risk factors in 2025.",
    expected_output=f"Full analysis report in bullet points summarizing '{topic}'.",
    agent=researcher
)

write_task = Task(
    description=f"Using the insights provided, develop an engaging and structured report on '{topic}' that highlights the headwinds the S&P 500 index faces using research findings.",
    expected_output=f"A markdown summary of '{topic}' with clear sections.",
    agent=writer,
    context=[research_task]
)

# Define the critic task for the Critic agent.
critic_task = Task(
    description=critic_description,
    expected_output=critic_expected_output,
    agent=critic_agent,
    context=[write_task]  # This task uses the output of the write_task as context
)

# Define the rewriting task for the Rewriter agent.
rewriter_task = Task(
    description=rewriter_description,
    expected_output=rewriter_expected_output,
    agent=rewriter_agent,
    context=[research_task, write_task, critic_task]  # This task uses both the original report and the critique as context
)

# Set up the LLM-based manager for the hierarchical process.
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# Assemble the crew with all four agents and tasks.
research_crew = Crew(
    agents=[researcher, writer, critic_agent, rewriter_agent],
    tasks=[research_task, write_task, critic_task, rewriter_task],
    process=Process.hierarchical,  # Executes tasks using hte hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

KeyError: 'OPENAI_API_KEY'

# Hierarchical

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

# --------------------------
# Configuration for a generic research task
# --------------------------
research_topic = "What are the implications for Nvidia on the Deepseek model?"
research_description = (
    f"Research the topic '{research_topic}'. Provide a summary of key findings with supporting details."
)
research_expected_output = (
    f"A collection of bullet points detailing the major insights about '{research_topic}', including relevant sources."
)

writing_description = (
    f"Compose an analysis report summarizing the findings on '{research_topic}'. "
    "The report should be structured, easy to read, and include clear headings and bullet points."
)
writing_expected_output = (
    f"A well-structured markdown summary of research findings on '{research_topic}', with clear headings and references."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()        # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

# --------------------------
# Create the Researcher agent, whose role is to gather information from the web.
# --------------------------
researcher_agent = Agent(
    name="Researcher",
    role="Research Analyst",
    goal="Gather reliable information on any given topic",
    backstory="An expert researcher adept at finding and curating information from diverse sources on the web.",
    tools=[search_tool, web_reader_tool],
    verbose=True
)

# --------------------------
# Create the Writer agent, whose role is to compile a summary report based on research.
# --------------------------
writer_agent = Agent(
    name="Writer",
    role="Content Writer",
    goal="Summarize research findings into a clear report",
    backstory="A skilled writer who can explain complex information in simple terms.",
    tools=[],
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the writing/summarization task for the Writer agent.
# --------------------------
write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # This task uses the output of the research_task as context
)

# --------------------------
# Set up the LLM-based manager for the hierarchical process.
# --------------------------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# --------------------------
# Assemble the crew with both agents and tasks using a hierarchical process.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent],
    tasks=[research_task, write_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final analysis report as markdown
display(Markdown(result.raw))


### Include critic

In [ ]:
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display

# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

# --------------------------
# Configuration for a generic research task
# --------------------------
topic = "What are JPM and GS 2025 outlook for the S&P 500 index?"

critic_description = (
    f"Review the financial analysis report on the outlook of the S&P 500 and summarizing the findings on '{topic}'. "
    "Provide detailed feedback on the clarity, coherence, and overall quality of the report, "
    "including strengths, weaknesses, and suggestions for improvement."
)
critic_expected_output = (
    "A collection of bullet points or paragraphs providing constructive critique of the analysis report, "
    "highlighting strengths, weaknesses, and potential improvements."
)

rewriter_description = (
    f"Rewrite the analysis report on '{topic}' by incorporating the feedback from the Critic. "
    "Produce a refined version that addresses the criticisms and improves clarity and structure."
)
rewriter_expected_output = (
    f"A refined markdown analysis report on '{topic}', incorporating the critic's feedback, "
    "with clear headings and detailed explanations."
)

# --------------------------
# Set up the tools for the Researcher agent: a web search tool and a web content reader.
# --------------------------
search_tool = SerperDevTool()        # Enables web searching via the Serper API
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

# Create the Researcher agent whose role is to gather info from the web and PDFs
researcher = Agent(
    name="Senior Financial Researcher",
    role="Financial Research Analyst",
    goal="Uncover insights about the 2025 outlook of the S&P 500 index",
    backstory="""You work at an asset management firm.
  Your goal is to understand the equity markets and have insight about the future outlook of the S&P 500.""",
    tools=[SerperDevTool(), WebsiteSearchTool(), query_tool],
    verbose=True
)

# Create the Writer agent
writer = Agent(
    name="Financial Writer",
    role="Financial Content Strategist",
    goal="Craft compelling content on the equity market and the future outlook of the S&P 500",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True
)


# --------------------------
# Create the Critic agent, whose role is to provide detailed feedback on the report.
# --------------------------
critic_agent = Agent(
    name="Critic",
    role="Content Critic",
    goal="Evaluate and provide constructive feedback on reports",
    backstory="An experienced critic with a keen eye for detail, focused on enhancing clarity, structure, and depth in reports.",
    tools=[],
    verbose=True
)

# --------------------------
# Define the research task for the Researcher agent.
# --------------------------
research_task = Task(
    description=research_description,
    expected_output=research_expected_output,
    agent=researcher_agent
)

# --------------------------
# Define the initial writing task for the Writer agent.
# --------------------------
initial_write_task = Task(
    description=writing_description,
    expected_output=writing_expected_output,
    agent=writer_agent,
    context=[research_task]  # Uses the output of the research task as context
)

# --------------------------
# Define the critique task for the Critic agent.
# --------------------------
critique_task = Task(
    description=critique_description,
    expected_output=critique_expected_output,
    agent=critic_agent,
    context=[initial_write_task]  # Uses the initial report as context
)

# --------------------------
# Define the rewrite task for the Writer agent to update the report based on the critique.
# --------------------------
rewrite_task = Task(
    description=rewrite_description,
    expected_output=rewrite_expected_output,
    agent=writer_agent,
    context=[research_task, initial_write_task, critique_task]  # Uses both research and critique feedback as context
)

# --------------------------
# Set up the LLM-based manager for the hierarchical process.
# --------------------------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# --------------------------
# Assemble the crew with all agents and tasks using a hierarchical process.
# --------------------------
research_crew = Crew(
    agents=[researcher_agent, writer_agent, critic_agent],
    tasks=[research_task, initial_write_task, critique_task, rewrite_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,         # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# Execute the crew's tasks
result = research_crew.kickoff()

# Render the final updated analysis report as markdown
display(Markdown(result.raw))
